In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/Colab Notebooks/PDS/train.csv" "./data.csv"

In [4]:
df = pd.read_csv('./data.csv')

print("Original DataFrame:")
print(df)
print("\n" + "="*50 + "\n")

Original DataFrame:
      Unnamed: 0                              Name    Location  Year  \
0              1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1              2                      Honda Jazz V     Chennai  2011   
2              3                 Maruti Ertiga VDI     Chennai  2012   
3              4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4              6            Nissan Micra Diesel XV      Jaipur  2013   
...          ...                               ...         ...   ...   
5842        6014                  Maruti Swift VDI       Delhi  2014   
5843        6015          Hyundai Xcent 1.1 CRDi S      Jaipur  2015   
5844        6016             Mahindra Xylo D4 BSIV      Jaipur  2012   
5845        6017                Maruti Wagon R VXI     Kolkata  2013   
5846        6018             Chevrolet Beat Diesel   Hyderabad  2011   

      Kilometers_Driven Fuel_Type Transmission Owner_Type     Mileage  \
0                 41000    Diesel       Ma

In [5]:
print("Missing values before handling:")
print(df.isnull().sum())
print("\n")

Missing values before handling:
Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64




In [6]:
cols_with_missing = df.columns[df.isnull().any()].tolist()

for col in cols_with_missing:
    if col == 'New_Price':
        df[col].fillna(0, inplace=True)
        print(f"Replacing 0 price for missing values in New_Price")
    elif df[col].dtype == 'object':
        most_frequent = df[col].mode()[0]
        df[col].fillna(most_frequent, inplace=True)
        print(f"Populating missing values in '{col}' with mode: '{most_frequent}'")
    elif df[col].dtype in ['int64', 'float64']:
        median_value = df[col].median()
        df[col].fillna(median_value, inplace=True)
        print(f"Populating missing values in '{col}' with median: {median_value}")

print("\nMissing values after handling:")
print(df.isnull().sum())
print("\n" + "="*50 + "\n")

Imputed missing values in 'Mileage' with mode: '18.9 kmpl'
Imputed missing values in 'Engine' with mode: '1197 CC'
Imputed missing values in 'Power' with mode: '74 bhp'
Imputed missing values in 'Seats' with median: 5.0
Replacing 0 price for missing values in New_Price

Missing values after handling:
Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64




<ipython-input-6-d1666b7966ad>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(most_frequent, inplace=True)
<ipython-input-6-d1666b7966ad>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df

Reason for Missing Value Handling:
- Choosing to fill missing data rather than simply removing rows helps preserve a broader dataset, which is especially important if the dataset isn't exceptionally large. Deleting rows could lead to significant information loss and skew the analysis.
- For category-based features like 'Owner_Type' and 'Location', using the mode to replace missing values is logical. It substitutes with the most common category, which helps to maintain the original data distribution.
- When dealing with numerical features such as 'Mileage' or 'Price', the median is preferable to the mean because it's less sensitive to extreme values. This ensures that the replacement values are more representative.
- The decision to impute was made to maximize the data available for analysis. The method of imputation was tailored to each column's data type.

In [8]:
cols_to_clean = ['Mileage', 'Engine', 'Power', 'New_Price']
for col in cols_to_clean:
    if col in df.columns:
        df[col] = df[col].astype(str).str.extract(r'(\d+\.?\d*)').astype('float')
        print(f"Removed units from column '{col}'")

print("\nDataFrame after removing units:")
print(df)
print("\n" + "="*50 + "\n")

Removed units from column 'Mileage'
Removed units from column 'Engine'
Removed units from column 'Power'
Removed units from column 'New_Price'

DataFrame after removing units:
      Unnamed: 0                              Name    Location  Year  \
0              1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1              2                      Honda Jazz V     Chennai  2011   
2              3                 Maruti Ertiga VDI     Chennai  2012   
3              4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4              6            Nissan Micra Diesel XV      Jaipur  2013   
...          ...                               ...         ...   ...   
5842        6014                  Maruti Swift VDI       Delhi  2014   
5843        6015          Hyundai Xcent 1.1 CRDi S      Jaipur  2015   
5844        6016             Mahindra Xylo D4 BSIV      Jaipur  2012   
5845        6017                Maruti Wagon R VXI     Kolkata  2013   
5846        6018             Che

In [9]:
categorical_cols = ['Fuel_Type', 'Transmission']

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
print("DataFrame after one-hot encoding:")
print(df)
print("\n" + "="*50 + "\n")

DataFrame after one-hot encoding:
      Unnamed: 0                              Name    Location  Year  \
0              1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1              2                      Honda Jazz V     Chennai  2011   
2              3                 Maruti Ertiga VDI     Chennai  2012   
3              4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4              6            Nissan Micra Diesel XV      Jaipur  2013   
...          ...                               ...         ...   ...   
5842        6014                  Maruti Swift VDI       Delhi  2014   
5843        6015          Hyundai Xcent 1.1 CRDi S      Jaipur  2015   
5844        6016             Mahindra Xylo D4 BSIV      Jaipur  2012   
5845        6017                Maruti Wagon R VXI     Kolkata  2013   
5846        6018             Chevrolet Beat Diesel   Hyderabad  2011   

      Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  New_Price  \
0                 41

In [10]:
current_year = 2025
df['Car_Age'] = current_year - df['Year']
print("DataFrame with 'Car_Age' feature:")
print(df)
print("\n" + "="*50 + "\n")

DataFrame with 'Car_Age' feature:
      Unnamed: 0                              Name    Location  Year  \
0              1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1              2                      Honda Jazz V     Chennai  2011   
2              3                 Maruti Ertiga VDI     Chennai  2012   
3              4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4              6            Nissan Micra Diesel XV      Jaipur  2013   
...          ...                               ...         ...   ...   
5842        6014                  Maruti Swift VDI       Delhi  2014   
5843        6015          Hyundai Xcent 1.1 CRDi S      Jaipur  2015   
5844        6016             Mahindra Xylo D4 BSIV      Jaipur  2012   
5845        6017                Maruti Wagon R VXI     Kolkata  2013   
5846        6018             Chevrolet Beat Diesel   Hyderabad  2011   

      Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  New_Price  \
0                 41

In [12]:
selected_cols_df = df[['Name', 'Location', 'Car_Age', 'Kilometers_Driven', 'Price']]
print("SELECT operation (selected columns):")
print(selected_cols_df)
print("\n")

filtered_df = df[df['Fuel_Type_Petrol'] == 1]
print("FILTER operation (Petrol cars):")
print(filtered_df)
print("\n")

renamed_df = df.rename(columns={'Kilometers_Driven': 'Driven_Kms', 'Car_Age': 'Age_in_Years'})
print("RENAME operation (renamed columns):")
print(renamed_df.head())
print("\n")

SELECT operation (selected columns):
                                  Name    Location  Car_Age  \
0     Hyundai Creta 1.6 CRDi SX Option        Pune       10   
1                         Honda Jazz V     Chennai       14   
2                    Maruti Ertiga VDI     Chennai       13   
3      Audi A4 New 2.0 TDI Multitronic  Coimbatore       12   
4               Nissan Micra Diesel XV      Jaipur       12   
...                                ...         ...      ...   
5842                  Maruti Swift VDI       Delhi       11   
5843          Hyundai Xcent 1.1 CRDi S      Jaipur       10   
5844             Mahindra Xylo D4 BSIV      Jaipur       13   
5845                Maruti Wagon R VXI     Kolkata       12   
5846             Chevrolet Beat Diesel   Hyderabad       14   

      Kilometers_Driven  Price  
0                 41000  12.50  
1                 46000   4.50  
2                 87000   6.00  
3                 40670  17.74  
4                 86999   3.50  
...     

In [15]:
df['Price_Per_KM'] = df['Price'] / (df['Kilometers_Driven'] + 1e-6) # Adding a small epsilon to avoid division by zero
print("MUTATE operation (added 'Price_Per_KM'):")
print(df[['Name', 'Price', 'Kilometers_Driven', 'Price_Per_KM']].head())
print("\n")

arranged_df = df.sort_values(by='Car_Age', ascending=False)
print("ARRANGE operation (sorted by 'Car_Age' descending):")
print(arranged_df[['Name', 'Car_Age']].head())
print("\n")

grouped_df = df.groupby('Fuel_Type_Petrol')['Price'].agg(['mean', 'median', 'count']).reset_index()
grouped_df['Fuel_Type'] = grouped_df['Fuel_Type_Petrol'].apply(lambda x: 'Petrol' if x == 1 else 'Diesel')
print("SUMMARIZE with GROUP BY ('Fuel_Type'):")
print(grouped_df[['Fuel_Type', 'mean', 'median', 'count']])

MUTATE operation (added 'Price_Per_KM'):
                               Name  Price  Kilometers_Driven  Price_Per_KM
0  Hyundai Creta 1.6 CRDi SX Option  12.50              41000      0.000305
1                      Honda Jazz V   4.50              46000      0.000098
2                 Maruti Ertiga VDI   6.00              87000      0.000069
3   Audi A4 New 2.0 TDI Multitronic  17.74              40670      0.000436
4            Nissan Micra Diesel XV   3.50              86999      0.000040


ARRANGE operation (sorted by 'Car_Age' descending):
                                   Name  Car_Age
3630  Mercedes-Benz E-Class 250 D W 210       27
5558                      Maruti Zen LX       27
3039                     Maruti Zen LXI       27
1791                 Honda City 1.3 EXI       26
1185                      Maruti Zen VX       26


SUMMARIZE with GROUP BY ('Fuel_Type'):
  Fuel_Type       mean  median  count
0    Diesel  12.960632   7.850   3163
1    Petrol   5.756688   4.205   2684
